# My hangman file

In [56]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
import pandas as pd
import numpy as np
import torch

In [46]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


Hmmmm, I should do a test-train split on the full dictionary...

In [47]:
from sklearn.model_selection import train_test_split
dict_train, dict_test = train_test_split(full_dict)

ughhhh how do I actually code up the AI model? I have the GAME, I need the PLAYER!


The model needs to accept:
an n-letter word, with letters/blanks encoded as one-hots (n by 27 dense vector)
a 1 x 26 vector of guesses made so far (0 for no guess, 1 for guess)

it needs to return as output:
1 x 26 vector of which argmax is the letter the model "chooses next"

In [48]:
n_epochs = 100
max_input_length = max([len(word) for word in dict_train])
batch_size = np.array([len(word) for word in dict_train]).mean()
print("Training set size: %i" % len(dict_train))
print("Max input size: %i" % max_input_length)
print("Average word length: %f" % batch_size)


Training set size: 170475
Max input size: 29
Average word length: 9.354416


In [49]:
max_input_length

29

In [50]:
len(full_dict)

227300

In [51]:
alphabet = list(string.ascii_lowercase)
orig_word = random.choice(dict_train)
blanked_word = "_" * len(orig_word)
#I want a distribution biased towards 0 instead of a flat one for choosing guesses
#do I though?
num_guesses = round(random.triangular(0,10))
guess_list = random.sample(alphabet, num_guesses)

if(num_guesses != 0):
    for guess_letter in guess_list:
        indices = [i for i, x in enumerate(list(orig_word)) if x == guess_letter]
        wordList = list(blanked_word)
        for i in indices:
            wordList[i] = guess_letter #now I replace the occurrences of the letter
        blanked_word = "".join(wordList)

remaining_correct_guesses = list(set(orig_word) - set(guess_list))

print(orig_word)
print(blanked_word)
print(guess_list)
print(remaining_correct_guesses)


theopolitics
_h_____i_i__
['i', 'r', 'm', 'x', 'h', 'w']
['e', 'c', 'o', 'l', 't', 's', 'p']


Ok, so now what, we have input:

blanked word -> encode it
guess list -> encode it
remaining correct guesses *this is the target -> encode it?

how do I encode the blanked word though? the strings also have variable length...

Ok, convert word into numbers

In [65]:
def letter_list_to_number(letter_list):
        #this takes a list of letters and returns a list of 
        out = []
        alphabet = list(string.ascii_lowercase)
        for letter in letter_list:
                if letter == "_":
                        out.append(27) #putting this here so I can also use it on the hidden word
                else:
                    out.append(alphabet.index(letter))
        return out

guess_list_encoded = letter_list_to_number(guess_list)
orig_word_encoded = letter_list_to_number(list(orig_word))
word = [i if i in guess_list_encoded else 26 for i in orig_word_encoded]
obscured_word = np.zeros((len(word), 27), dtype=np.float32)
print(word)
for i, j in enumerate(word):
            #print(i,j)
            obscured_word[i, j] = 1
#print(obscured_word)

[26, 26, 26, 26]


In [66]:
print(obscured_word.shape)

(4, 27)


In [67]:
guess_list_encoded = letter_list_to_number(guess_list)
orig_word_encoded = letter_list_to_number(list(orig_word))
print(guess_list_encoded)
print(orig_word_encoded)

[3, 25, 6, 16]
[13, 8, 12, 1]


In [55]:
def letter_list_to_number(letter_list):
        out = np.zeros(len(letter_list))
        alphabet = list(string.ascii_lowercase)
        for letter in letter_list:
                if letter == "_":
                        out.append(27) #putting this here so I can also use it on the hidden word
                else:
                    np.append(out,alphabet.index(letter))
        return out
                
guess_list_encoded = letter_list_to_number(guess_list)
print(guess_list_encoded)

[0. 0. 0. 0. 0. 0.]


In [ ]:
def encode_obscured_word(self):
        word = [i if i in self.letters_guessed else 26 for i in self.full_word]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word):
            obscured_word[i, j] = 1
        return(obscured_word)

In [42]:
list(set(orig_word))

['n', 'r', 'e', 'l', 'i', 'y', 'a']

In [41]:
set(guess_list)

{'b', 'r', 't', 'y'}

In [43]:
remaining_correct_guesses = list(set(orig_word) - set(guess_list))

['n', 'e', 'l', 'i', 'a']

Ok... I can't feed in the letters one at a time... bummer. I need to pad it out

Since we are feeding stuff to torch, stuff has to be in the form n x 1 x 27, since it assumes we are working in batches?

In [58]:
n_letters = len(alphabet)+1 #+1 for underscore

In [59]:
# n_letters needs to be 27 to include underscore
alphabet = list(string.ascii_lowercase)

def letter_to_number(letter):
    if letter == "_": #this is a blank space
        return 27
    if letter == "0" #this is padding
    else:
        return alphabet.index(letter)


# Just for demonstration, turn a letter into a <1 x n_letters+1> Tensor
def letter_to_tensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letter_to_number(letter)] = 1
    return tensor

# Turn a line into a <word_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def word_to_tensor(word):
    tensor = torch.zeros(len(word), 1, n_letters)
    for widx, letter in enumerate(word):
        tensor[widx][0][letter_to_number(letter)] = 1
    return tensor

print(letter_to_tensor('a'))

print(word_to_tensor('apple').size())

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([5, 1, 27])


In [61]:
alphabet = list(string.ascii_lowercase)
orig_word = random.choice(dict_train)
blanked_word = "_" * len(orig_word)
#I want a distribution biased towards 0 instead of a flat one for choosing guesses
#do I though?
num_guesses = round(random.triangular(0,6))
guess_list = random.sample(alphabet, num_guesses)

if(num_guesses != 0):
    for guess_letter in guess_list:
        indices = [i for i, x in enumerate(list(orig_word)) if x == guess_letter]
        wordList = list(blanked_word)
        for i in indices:
            wordList[i] = guess_letter #now I replace the occurrences of the letter
        blanked_word = "".join(wordList)

remaining_correct_guesses = list(set(orig_word) - set(guess_list))

print(orig_word)
print(blanked_word)
print(guess_list)
print(remaining_correct_guesses)


nimb
____
['d', 'z', 'g', 'q']
['n', 'b', 'm', 'i']


In [62]:
max_input_length

29

In [ ]:
longest_word_length = 45

In [69]:
def letter_to_number(letter_list):
        #this takes a list of letters and returns a list of 
        out = []
        alphabet = list(string.ascii_lowercase)
        for letter in letter_list:
                if letter == "_":
                        out.append(27) #putting this here so I can also use it on the hidden word
                else:
                    out.append(alphabet.index(letter))
        return out

guess_list_encoded = letter_list_to_number(guess_list)
orig_word_encoded = letter_list_to_number(list(orig_word))
word = [i if i in guess_list_encoded else 26 for i in orig_word_encoded]
obscured_word = np.zeros((len(word), 27), dtype=np.float32)
print(word)
for i, j in enumerate(word):
            #print(i,j)
            obscured_word[i, j] = 1
print(obscured_word)

[26, 26, 26, 26]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1.]]


In [73]:
alphabet.append("_","=")
print(alphabet)

TypeError: list.append() takes exactly one argument (2 given)

In [104]:
def word_to_tensor(current_word):
    longest_word_length = 45 #this is the longest word in the english language
    #first, encode word: ex "apple"
    #assume word is something like "_pp__"
    alphabet = list(string.ascii_lowercase) 
    alphabet.append("_")#this is for blanks
    #alphabet.append("=") # using this for generic padding
    #input_length = longest_word_length + n_letters+1
    num_letters = len(alphabet)

    # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
    # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
    current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
    for widx, letter in enumerate(current_word):
        #print(widx)
        #print(letter)
        current_word_tensor[0][widx][alphabet.index(letter)] = 1
    #print(current_word_tensor)
    return current_word_tensor

def guess_list_to_tensor(guess_list):
    #need to encode the guess list as a 1 x 26 tensor
    guess_tensor = torch.zeros(1, 1, 26)
    for letter in guess_list:
        guess_tensor[0][0][alphabet.index(letter)] = 1
    
    return guess_tensor

    

In [106]:
print(word_to_tensor("_pp__").shape)
print(guess_list_to_tensor(["p"]).shape)

torch.Size([1, 27, 45])
torch.Size([1, 1, 26])


Ok, so the current word is converted into a tensor and then fed into a LSTM, the guess list is converted to a tensor and then COMBINED with the output from the LSTM and then that combo is fed into a dense layer.

In [110]:
import torch.nn as nn

class Player(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Player, self).__init__()
        self.num_classes = num_classes #number of outputs
        self.num_layers = num_layers #I think this is arbitrary
        self.input_size =  input_size #input features
        self.hidden_size = hidden_size #to match number of letters in encoded guess
        self.seq_length = seq_length #to match number of letters in input
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size * 2, 128) #fully connected 1
        self.fc = nn.Linear(128, 26) #fully connected last layer

    
    def forward(self, encoded_word, encoded_guess):
        h_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #hidden state
        c_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(encoded_word, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        hn = torch.cat(hn,encoded_guess) #combine data with encoded guess data before passing to dense layer

        out = self.fc_1(out) #first Dense

        out = self.fc(out) #Final Output
        return out

In [ ]:
num_classes = len(alphabet)-1
num_layers = 10 #I think this is arbitrary
input_size =  len(alphabet)
hidden_size = len(alphabet)-1
seq_length = longest_word_length

In [112]:
longest_word_length = 45
player1 = Player(num_classes = len(alphabet)-1, num_layers = 10,
                input_size =  len(alphabet),
                hidden_size = len(alphabet)-1,
                seq_length = longest_word_length)

In [ ]:
import torch.nn as nn

class Player(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Player, self).__init__()

        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
        self.h2o = nn.Linear(hidden_size, output_size)
        
        self.softmax = nn.LogSoftmax(dim=1)
        self.guess_tensor = torch.zeros(1, 1, 26)

    def forward(self, word_tensor, guess_tensor):
        """
        forward takes word_tensor and 
        """

        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = torch.cat(self.h2o(hidden), guess_tensor)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
    def get_guess_tensor(self, new_gt):
        self.guess_tensor = new_gt

In [ ]:
def create_LSTM_net(input_obscured_word_seen, input_letters_guessed_previously):
    with cntk.layers.default_options(initial_state = 0.1):
        lstm_outputs = cntk.layers.Recurrence(cntk.layers.LSTM(MAX_NUM_INPUTS))(input_obscured_word_seen)
        final_lstm_output = cntk.ops.sequence.last(lstm_outputs)
        combined_input = cntk.ops.splice(final_lstm_output, input_letters_guessed_previously)
        dense_layer = cntk.layers.Dense(26, name='final_dense_layer')(combined_input)
        return(dense_layer)
    
input_obscured_word_seen = cntk.ops.input_variable(shape=27,
                                                   dynamic_axes=[cntk.Axis.default_batch_axis(),
                                                                 cntk.Axis.default_dynamic_axis()],
                                                   name='input_obscured_word_seen')
input_letters_guessed_previously = cntk.ops.input_variable(shape=26,
                                                           dynamic_axes=[cntk.Axis.default_batch_axis()],
                                                           name='input_letters_guessed_previously')

z = create_LSTM_net(input_obscured_word_seen, input_letters_guessed_previously)

In [78]:
alphabet = list(string.ascii_lowercase)

In [79]:
print(alphabet)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
import torch.nn as nn
#input_length = longest_word_length + n_letters+1

# Size: [batch_size, seq_len, input_size]
input = torch.randn(1, longest_word_length, n_letters)

lstm = nn.LSTM(input_size=input_length, hidden_size=512, batch_first=True)

output, _ = lstm(input)
output.size()  # => torch.Size([12, 384, 512])

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(2 * n_letters, n_hidden, n_letters)

What if the AI attempts to guess a letter that has been chosen? Give a penalty?

The clustering here is useless. The words are not grouped together in a way that they are close in terms of guessing in hangman. I have to figure out my own distance metric for this and I don't have time.

In [30]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
    
#words = "YOUR WORDS HERE".split(" ") #Replace this line
#words = dict_train

#lets try a small subset of dict_train first to make sure this works.....
words = random.sample(dict_train, round(len(dict_train)/100))

words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])
lev_similarity = np.asarray(lev_similarity)

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)

affprop.fit(lev_similarity.astype(float))
#affprop.fit(words)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *hyperlogically:* hyperlogically, hypertrichy, nonhypostatically, supercatholically
 - *tyroid:* assyroid, boronia, gyroma, hydrophis, pygmoid, rhizoid, syconid, tetraiodid, thoom, throes, thrombosed, throwforward, torpidity, tyburnian, tyroid
 - *multicrystalline:* multicrystalline, transcrystalline
 - *unlatched:* tilthead, unclutched, unjapanned, unlatched, unlimned, unmature, unskaithed
 - *noninterchangeable:* noninterchangeable
 - *superfluitance:* superfluitance, superincumbency
 - *youngmannishness:* youngmannishness
 - *carcinosarcomata:* carcinosarcomata
 - *greatgrandparent:* greatgrandparent
 - *pims:* dump, fizz, jmx, limosi, mim, peasy, picco, pijaw, pims, plied, psocids, qms, quiffs, rdbms, simpai, smiris, tpmp, tws, updives, wights
 - *cystoepiplocele:* cystoepiplocele, glossoepiglottic
 - *moodys:* anotus, broadways, houndish, mobby, monkism, monogyny, moocha, moodir, moodys, myotomes, topotypes
 - *cryptoprotestant:* cryptoprotestant
 - *deeptangled:* deeptangled, 

In [31]:
affprop.labels_

array([182,  11,   2, ...,  52,  36, 117], dtype=int64)

In [26]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
    
#words = "YOUR WORDS HERE".split(" ") #Replace this line
#words = dict_train

#lets try a small subset of dict_train first to make sure this works.....
words = random.sample(dict_train, round(len(dict_train)/1000))

words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

In [24]:
lev_similarity

array([[  0, -13, -12, ..., -13, -12, -12],
       [-13,   0, -11, ..., -10, -10, -13],
       [-12, -11,   0, ...,  -8,  -8,  -9],
       ...,
       [-13, -10,  -8, ...,   0, -10, -13],
       [-12, -10,  -8, ..., -10,   0,  -9],
       [-12, -13,  -9, ..., -13,  -9,   0]])

In [20]:
from sklearn_extra.cluster import KMedoids

ModuleNotFoundError: No module named 'sklearn_extra'

In [15]:
len(dict_train)/100

1704.75

So, I think the best way to do this would be to make a decision tree from scratch, but I think there is an issue of outliers that I am not super sure how to deal with. I think the dictionary is big enough that there are some words that the algorithm should figure out how to throw out. Like if there are 2 moves left, it should make the highest probability assumption it can which eliminates trees with more than 2 moves of surprise.

So... I think how the algorithm should work is, it doesnt just pick the ONE letter that would maximize the entropy next, it should find like a PATH of guesses that maximally reduces the entropy.

I am assuming there are always 6 tries, which seems a bit low...

It seems a bit complex though... If I build a tree I don't have to constantly re-calculate it, but I just move from node to node.

HOW DO I MAKE A DECISION TREE CLASSIFIER WITH NO LABELS THOUGH?

For testing purposes, I want to just remake hangman here so I can mess with it. So gut all of the connection to some external anything. I think I know how it works from running it a bit.

In [15]:
class MyHangman2(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.has_won = False
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)

    def get_winstate(self):
        return self.has_won


    def guess(self, word): # word input example: "_ p p _ e "
        all_letters = [x[0] for x in self.full_dictionary_common_letter_sorted]


        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".")
        #print(word)
        #print(word[::2])
        #print(clean_word)

        # find length of passed word
        len_word = len(clean_word)

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.full_dictionary

        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            #NOTE THIS MAY NOT BE HOW I DO THIS AFTER I DO THE STUFF BELOW!

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        current_dictionary = new_dictionary


        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)                 

        guess_letter = '!'


        # what options do we still have to guess?
        letter_options = [letter for letter in all_letters if letter not in self.guessed_letters]

        # break down the possibilities by letter if that letter is correct
        # future_guess_array = []
        # guess_entropy_array = []

        def calc_entropy(base, subset): 
            # This calculates the entropy of taking a set A and breaking it down into a subset B and B complement
            # I mean mathematically, this acts on lists rather sets programming wise.

            #I have domain issues if p=1 or p=0, so if the base == subset or if subset empty
            if base == subset: #hmmmmmm how do I handle this?
                return 1
            elif len(subset) == 0:
                return 0
            else:
                p = len(subset) / len(base)
                return -(p*math.log(p) + (1-p)*math.log(1-p))
                
        # for each letter, calculate the entropy of choosing that letter
        word_count_total = len(current_dictionary)
        guess_entropy_array = []

        for letter in letter_options:
            #dict_by_letter = []
            word_count_by_letter = 0

            # This process here is very slow, for each letter it counts how many words in the dictionary have that letter present.
            # I think there is a better way to do this...
            for word in current_dictionary:
                if set(letter) & set(word): # I use sets to get all the words with a specific letter
                    #dict_by_letter.append(word)
                    word_count_by_letter+=1
            
            
            p = word_count_by_letter / word_count_total


            # p=0 and p=1 are special and need to treat them differently, they are out of the domain of log
            # p=0 means there are no words remaining in the dictionary, so that guess is useless and I set the entropy to 0
            # p=1 will only occur when the number of words corresponding to a letter is the same as in the entire dictionary
            # this will only occur when we have locked in on one possible answer, so I set the entropy to 1 which is greater than the normal max the entropy can have

            if p == 0:
                letter_entropy = 0
            elif p == 1:
                letter_entropy = 1
            else:
                letter_entropy = -(p*math.log(p) + (1-p)*math.log(1-p))
            #future_guess_array.append(dict_by_letter)
            #guess_entropy = calc_entropy(current_dictionary, dict_by_letter)
            #guess_entropy_array.append(guess_entropy)
            guess_entropy_array.append(letter_entropy)

        # now, which letter had the highest entropy?
        idx_max = guess_entropy_array.index(max(guess_entropy_array))
        best_letter = letter_options[idx_max]
        guess_letter = best_letter
                    
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        self.has_won = False
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                


            



        """""           
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
    """""

In [16]:
game = MyHangman2()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: ________.
Guessing letter: r
Got a Letter, r, # of tries remaining: 6. Word: r_______.
Guessing letter: a
Got a Letter, a, # of tries remaining: 6. Word: r_a_____.
Guessing letter: s
Incorrect guess s, # of tries remaining: 6. Word: r_a_____.
Guessing letter: d
Got a Letter, d, # of tries remaining: 5. Word: r_ad____.
Guessing letter: o
Got a Letter, o, # of tries remaining: 5. Word: road____.
Guessing letter: e
Got a Letter, e, # of tries remaining: 5. Word: road___e.
Guessing letter: i
Got a Letter, i, # of tries remaining: 5. Word: road_i_e.
Guessing letter: l
Got a Letter, l, # of tries remaining: 5. Word: roadli_e.
Guessing letter: k
Success! the word was: roadlike


In [47]:
game.get_winstate()

True

In [17]:
N = 1000
wincount = 0
game = MyHangman2()
t = time.time()

for i in range(N):
    elapsed = time.time() - t
    if (i+1) % 100 == 0:
        print("On Game %i, elapsed time is %f, current winrate is %f" % (i+1, elapsed ,100*wincount/(i)))
    game.start_game(verbose = False)
    wincount += game.get_winstate()
    #if(game.get_winstate()):
    #    print("Won game %i, win count = %i, time elapsed = %f" % (i+1, wincount, elapsed))
print("Final Score: %f percent wins. Time elapsed: %f" % (100 * wincount / N, elapsed))

On Game 100, elapsed time is 85.118947, current winrate is 90.909091
On Game 200, elapsed time is 167.148824, current winrate is 89.447236
On Game 300, elapsed time is 250.472179, current winrate is 88.294314
On Game 400, elapsed time is 327.258479, current winrate is 89.223058
On Game 500, elapsed time is 412.823485, current winrate is 88.577154
On Game 600, elapsed time is 494.952882, current winrate is 86.811352
On Game 700, elapsed time is 572.957901, current winrate is 86.552217
On Game 800, elapsed time is 645.716178, current winrate is 86.357947
On Game 900, elapsed time is 730.106674, current winrate is 86.651835
On Game 1000, elapsed time is 806.408744, current winrate is 86.186186
Final Score: 86.200000 percent wins. Time elapsed: 806.408744


with word_count method, takes 8.2 secs for 10 games... 18 sec for 25 games

With the large lists, takes 9.0 secs for 10 games... 27 sec for 25 games

In [8]:
game = MyHangman3()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: ___________.
Guessing letter: n
Got a Letter, n, # of tries remaining: 6. Word: _n_________.
Guessing letter: l
Got a Letter, l, # of tries remaining: 6. Word: _n_______l_.
Guessing letter: r
Got a Letter, r, # of tries remaining: 6. Word: _n____r__l_.
Guessing letter: b
Got a Letter, b, # of tries remaining: 6. Word: _n____r_bl_.
Guessing letter: i
Incorrect guess i, # of tries remaining: 6. Word: _n____r_bl_.
Guessing letter: a
Got a Letter, a, # of tries remaining: 5. Word: _n____rabl_.
Guessing letter: y
Incorrect guess y, # of tries remaining: 5. Word: _n____rabl_.
Guessing letter: s
Got a Letter, s, # of tries remaining: 4. Word: _ns___rabl_.
Guessing letter: u
Got a Letter, u, # of tries remaining: 4. Word: uns__urabl_.
Guessing letter: e
Got a Letter, e, # of tries remaining: 4. Word: unse_urable.
Guessing letter: c
Success! the word was: unsecurable


In [10]:
for i in range(10):
    game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Incorrect guess s, # of tries remaining: 5. Word: _________.
Guessing letter: o
Got a Letter, o, # of tries remaining: 4. Word: _o_______.
Guessing letter: t
Got a Letter, t, # of tries remaining: 4. Word: to_______.
Guessing letter: r
Got a Letter, r, # of tries remaining: 4. Word: torr_____.
Guessing letter: i
Got a Letter, i, # of tries remaining: 4. Word: torr__i__.
Guessing letter: e
Got a Letter, e, # of tries remaining: 4. Word: torre_ie_.
Guessing letter: d
Got a Letter, d, # of tries remaining: 4. Word: torre_ied.
Guessing letter: f
Success! the word was: torrefied
Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Got a Letter, s, # of tries remaining: 5. Word: ______s_s.
Guessing letter:

In [2]:
full_dict

NameError: name 'full_dict' is not defined